In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
plt.style.use('seaborn')

In [ ]:
data = pd.read_csv('dataset_a_cleaned.csv', index_col=0)
speed = data['0'].values.flatten()
power = data['1'].values.flatten()
speed = speed[:50000]/max(speed[:50000])
power = power[:50000]/max(power[:50000])

In [ ]:
class Kernel:
    def kernel(self, a, b):
        norm = np.linalg.norm(a - b)
        term = (norm * norm) / (2 * self.sigma * self.sigma)
        return np.exp(-1 * term)
    
class KRLS(Kernel):
    def __init__(
        self,
        first_input,
        first_output,
        reg_param=0.1,
        sigma=1
    ):
        self.reg_param = reg_param
        self.sigma = sigma
        self.Q = np.array(
            [1 / (reg_param + self.kernel(first_input, first_input))])
        self.weights = np.array([self.Q * first_output])
        self.inputs = [first_input]

    def predict(self, new_input):
        h = np.array([self.kernel(new_input, old_input) for old_input in self.inputs]).reshape(1, len(self.inputs))
        return np.dot(h,self.weights)

    def update(self, new_input, expected):
        h = np.array([
            self.kernel(new_input, old_input) for old_input in self.inputs
        ]).reshape(len(self.inputs), 1)
        ht = h.T
        z = self.Q.dot(h)
        zt = z.T
        r = self.reg_param + self.kernel(new_input, new_input) - zt.dot(h)
        Q_size = len(z) + 1
        new_Q = np.zeros((Q_size, Q_size))
        new_Q[0:Q_size - 1, 0:Q_size - 1] = self.Q * r + z.dot(zt)
        new_Q[0:Q_size - 1, Q_size - 1:Q_size] = -z
        new_Q[Q_size - 1:Q_size, 0:Q_size - 1] = -zt
        new_Q[Q_size - 1][Q_size - 1] = 1
        self.Q = new_Q
        error = expected - ht.dot(self.weights)
        new_weights = np.zeros((Q_size, 1))
        new_weights[0:Q_size - 1, 0:1] = self.weights - z * (1 / r) * error
        new_weights[Q_size - 1][0] = (1 / r) * error
        self.weights = new_weights
        self.inputs.append(new_input)
        # reduce the regularization as we get more data
        self.reg_param *= 0.9


In [ ]:
llim = 0
ulim = 1000
step_size = 100
test_lim=10
regularisation = [0.1,0.01,0.001]
error_reg = []
plt.figure(figsize=[15,10])
for r in regularisation:
    error = []
    krlsFilter = KRLS(speed[0],power[0],sigma = 0.1, reg_param = r)
    for k in range(llim, ulim,step_size):
        for i in range(k,k+step_size):
            krlsFilter.update(speed[i],power[i])
        yhat = []
        for i in range(k+step_size,k+step_size+test_lim):
            yhat.append(krlsFilter.predict(speed[i])[0][0])
        yhat = np.array(yhat)
        error.append(mean_squared_error(power[k+step_size:k+step_size+test_lim],yhat)/test_lim)
        print("\r","Iteration = {} done...".format(k+step_size),end='')
    error = np.array(error)
    error_reg.append(error)
    plt.plot(error,label = "ref_param = {}".format(r))
    print("reg_param = {} completed".format(r))
plt.legend(loc='upper right')

In [ ]:
for i in range(len(error_reg)):
    pd.DataFrame(error_reg[i]).to_csv('krls_error_reg_{}.csv'.format(regularisation[i]))